**Does 10 predictions using XGBoost. Will spit out a csv of the ratios when run all the way through.**

In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
import matplotlib.pyplot as plt

In [ ]:
#McCullough Method
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
df = pd.read_csv(f"/home/jraskin3/vesta_csvs/32-probes-aligned-256-iters.csv").fillna(-1)
df = df[df.power <= 10**5]
df = df.drop([col for col in df.columns if '__entry' in col], axis=1)
df = df.drop([col for col in df.columns if '__return' in col], axis=1)
df = df.drop([col for col in df.columns if '__begin' in col], axis=1)
df = df.drop([col for col in df.columns if '__end' in col], axis=1)
df["cpu_power"] = (df["energy_component_2_power"] + df["energy_component_5_power"])
df["memory_power"] = (df["energy_component_0_power"] + df["energy_component_3_power"])
train_bench = "combined-benchmarks"
benchmarks = df.benchmark.unique()
ratios = {}
import tensorflow as tf


for bench in benchmarks:
    ratios[bench] = []
for i in range(0,10):
    for component in ["power"]:
        #model = XGBRegressor(tree_method='gpu_hist', gpu_id=0)
        #model = LinearRegression()
#         tf.config.threading.set_inter_op_parallelism_threads(2304)
#         nn_model = tf.keras.Sequential()
#         nn_model.add(tf.keras.layers.Dense(25, activation="tanh"))
#         nn_model.add(tf.keras.layers.Dense(50, activation="tanh"))
#         nn_model.add(tf.keras.layers.Dense(100, activation="tanh"))
#         nn_model.add(tf.keras.layers.Dense(50, activation="tanh"))
#         nn_model.add(tf.keras.layers.Dense(25, activation="tanh"))
#         nn_model.add(tf.keras.layers.Dense(1))
#         nn_model.compile(optimizer="sgd", loss='mse')
        events_train = pd.DataFrame()
        events_test = {}
        power_train = pd.DataFrame()
        power_test = {}
        for bench in benchmarks:
            events_df_train = df[df.benchmark == bench]
            power_df_train = events_df_train[component]

            events_df_train = events_df_train.drop(["iteration","ts","benchmark","power","cpu_power","memory_power"], axis=1)
            events_df_train = events_df_train.drop([col for col in events_df_train.columns if 'energy_component' in col], axis=1)
            if(len(events_df_train) <= 1):
                events_train = pd.concat([events_train, events_df_train])
                events_test[bench] = events_df_train
                power_train = pd.concat([power_train, power_df_train])
                power_test[bench] = power_df_train
                continue        

            events_train_b, events_test_b, power_train_b, power_test_b = train_test_split(events_df_train, power_df_train, test_size=.5)

            events_train = pd.concat([events_train, events_train_b])
            events_test[bench] = events_test_b

            power_train = pd.concat([power_train, power_train_b])
            power_test[bench] = power_test_b

        model.fit(events_train, power_train)
        #nn_model.fit(tf.convert_to_tensor(events_train), tf.convert_to_tensor(power_train), epochs=5, batch_size=256)
        for bench in benchmarks:
            prediction_energy = model.predict(events_test[bench])
            #prediction_energy = nn_model.predict(tf.convert_to_tensor(events_test[bench]), batch_size=256)
            real_energy = power_test[bench]
            total_prediction_energy = prediction_energy.sum()
            total_actual_energy = real_energy.sum()

            ratio =  np.abs((total_actual_energy - total_prediction_energy) / total_actual_energy)
            ratios[bench].append(ratio)

In [ ]:
ratio_mean = pd.DataFrame(ratios).mean()
ratio_mean_df = pd.DataFrame(ratio_mean).rename(columns={0:"mean"})
ratio_mean_df["std"] = pd.DataFrame(ratios).std()
ratio_mean_df = ratio_mean_df.sort_index()
ratio_mean_df = ratio_mean_df.reset_index(names=["benchmark"])
ratio_mean_df.to_csv("/home/jraskin3/vesta_csvs/32-probes-256-iters-LR-ratios.csv", index=False)

In [ ]:
len(ratio_mean_df[ratio_mean_df["mean"] <= .10])

In [ ]:
df = pd.read_csv(f"/home/jraskin3/vesta_csvs/perf-hpcs-aligned-256-iters.csv").fillna(-1)

In [ ]:
df.groupby("benchmark").count()["ts"].sort_values()

**All but one test**

In [ ]:
benches = benchmarks

In [ ]:
#McCullough Method
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
for the_bench in sorted(benches):
    df = pd.read_csv(f"/home/jraskin3/vesta_csvs/32-probes-aligned-256-iters.csv").fillna(-1)
    df = df[df.power <= 10**5]
    df = df.drop([col for col in df.columns if '__entry' in col], axis=1)
    df = df.drop([col for col in df.columns if '__return' in col], axis=1)
    df = df.drop([col for col in df.columns if '__begin' in col], axis=1)
    df = df.drop([col for col in df.columns if '__end' in col], axis=1)
    df["cpu_power"] = (df["energy_component_2_power"] + df["energy_component_5_power"])
    df["memory_power"] = (df["energy_component_0_power"] + df["energy_component_3_power"])
    train_bench = "combined-benchmarks"
    #the_bench = "biojava"
    benchmarks = df.benchmark.unique()
    benchmarks = set(benchmarks) - {the_bench}
    #print(sorted(benchmarks))
    ratios = {}
    import tensorflow as tf


    for bench in benchmarks:
        ratios[bench] = []
    for i in range(0,1):
        for component in ["power"]:
            model = XGBRegressor(tree_method='gpu_hist', gpu_id=0)
            #model = LinearRegression()
    #         tf.config.threading.set_inter_op_parallelism_threads(2304)
    #         nn_model = tf.keras.Sequential()
    #         nn_model.add(tf.keras.layers.Dense(25, activation="tanh"))
    #         nn_model.add(tf.keras.layers.Dense(50, activation="tanh"))
    #         nn_model.add(tf.keras.layers.Dense(100, activation="tanh"))
    #         nn_model.add(tf.keras.layers.Dense(50, activation="tanh"))
    #         nn_model.add(tf.keras.layers.Dense(25, activation="tanh"))
    #         nn_model.add(tf.keras.layers.Dense(1))
    #         nn_model.compile(optimizer="sgd", loss='mse')
            events_train = pd.DataFrame()
            events_test = {}
            power_train = pd.DataFrame()
            power_test = {}
            for bench in benchmarks:
                events_df_train = df[df.benchmark == bench]
                power_df_train = events_df_train[component]

                events_df_train = events_df_train.drop(["iteration","ts","benchmark","power","cpu_power","memory_power"], axis=1)
                events_df_train = events_df_train.drop([col for col in events_df_train.columns if 'energy_component' in col], axis=1)
                if(len(events_df_train) <= 1):
                    events_train = pd.concat([events_train, events_df_train])
                    events_test[bench] = events_df_train
                    power_train = pd.concat([power_train, power_df_train])
                    power_test[bench] = power_df_train
                    continue        

                events_train_b, events_test_b, power_train_b, power_test_b = train_test_split(events_df_train, power_df_train, test_size=.1)

                events_train = pd.concat([events_train, events_train_b])
                events_test[bench] = events_test_b

                power_train = pd.concat([power_train, power_train_b])
                power_test[bench] = power_test_b

            model.fit(events_train, power_train)
            #nn_model.fit(tf.convert_to_tensor(events_train), tf.convert_to_tensor(power_train), epochs=5, batch_size=256)
            for bench in benchmarks:
                prediction_energy = model.predict(events_test[bench])
                #prediction_energy = nn_model.predict(tf.convert_to_tensor(events_test[bench]), batch_size=256)
                real_energy = power_test[bench]
                total_prediction_energy = prediction_energy.sum()
                total_actual_energy = real_energy.sum()

                ratio =  np.abs((total_actual_energy - total_prediction_energy) / total_actual_energy)
                ratios[bench].append(ratio)
    bench_df = df[df.benchmark == the_bench]
    events_df_test = bench_df 
    events_df_test = events_df_test.drop(["iteration","ts","benchmark","power","cpu_power","memory_power"], axis=1)
    events_df_test = events_df_test.drop([col for col in events_df_test.columns if 'energy_component' in col], axis=1)
    prediction_energy = model.predict(events_df_test)
    real_energy = bench_df["power"]
    total_prediction_energy = prediction_energy.sum()
    total_actual_energy = real_energy.sum()

    ratio =  np.abs((total_actual_energy - total_prediction_energy) / total_actual_energy)
    print(the_bench, f"{ratio * 100:.02f}%")

**Real McCullough**

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [ ]:
#Faithful McCullough Method
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
df = pd.read_csv(f"/home/jraskin3/vesta_csvs/32-probes-aligned-256-iters.csv").fillna(-1)
df = df[df.power <= 10**5]
df = df.drop([col for col in df.columns if '__entry' in col], axis=1)
df = df.drop([col for col in df.columns if '__return' in col], axis=1)
df = df.drop([col for col in df.columns if '__begin' in col], axis=1)
df = df.drop([col for col in df.columns if '__end' in col], axis=1)
df["cpu_power"] = (df["energy_component_2_power"] + df["energy_component_5_power"])
df["memory_power"] = (df["energy_component_0_power"] + df["energy_component_3_power"])
train_bench = "combined-benchmarks"
benchmarks = df.benchmark.unique()
ratios = {}
import tensorflow as tf


for bench in benchmarks:
    ratios[bench] = []
for i in range(0,10):
    for component in ["power"]:
        #model = XGBRegressor(tree_method='gpu_hist', gpu_id=0)
        #model = LinearRegression()
#         tf.config.threading.set_inter_op_parallelism_threads(2304)
        nn_model = tf.keras.Sequential()
        nn_model.add(tf.keras.layers.Dense(25, activation="tanh"))
        #nn_model.add(tf.keras.layers.Dense(50, activation="tanh"))
        #nn_model.add(tf.keras.layers.Dense(50, activation="tanh"))
        nn_model.add(tf.keras.layers.Dense(12, activation="tanh"))
        nn_model.add(tf.keras.layers.Dense(1))
        nn_model.compile(optimizer="sgd", loss='mse')
        df_train, df_test = train_test_split(df, test_size=.5)
        events_df_train = df_train
        power_df_train = df_train[component]

        events_df_train = events_df_train.drop(["iteration","ts","benchmark","power","cpu_power","memory_power"], axis=1)
        events_df_train = events_df_train.drop([col for col in events_df_train.columns if 'energy_component' in col], axis=1)
        #model.fit(events_df_train, power_df_train)
        nn_model.fit(tf.convert_to_tensor(events_df_train), tf.convert_to_tensor(power_df_train), epochs=5, batch_size=256)
        for bench in benchmarks:
            events_test = df_test[df_test.benchmark == bench]
            if len(events_test) == 0:
                continue
            events_test = events_test.drop(["iteration","ts","benchmark","power","cpu_power","memory_power"], axis=1)
            events_test = events_test.drop([col for col in events_test.columns if 'energy_component' in col], axis=1)
            
            #prediction_energy = model.predict(events_test)
            prediction_energy = nn_model.predict(tf.convert_to_tensor(events_test), batch_size=256)
            real_energy = df_test[df_test.benchmark == bench]["power"]
            total_prediction_energy = prediction_energy.sum()
            total_actual_energy = real_energy.sum()

            ratio =  np.abs((total_actual_energy - total_prediction_energy) / total_actual_energy)
            ratios[bench].append(ratio)

In [ ]:
df = pd.read_csv(f"/home/jraskin3/vesta_csvs/32-probes-aligned-256-iters.csv").fillna(-1)
events_test = df.drop(["iteration","ts","benchmark","power"], axis=1)
events_test = events_test.drop([col for col in events_test.columns if 'energy_component' in col], axis=1)
events_test.columns

In [ ]:
ratio_mean = pd.DataFrame(ratios).mean()
ratio_mean_df = pd.DataFrame(ratio_mean).rename(columns={0:"mean"})
ratio_mean_df["std"] = pd.DataFrame(ratios).std()
ratio_mean_df = ratio_mean_df.sort_index()
ratio_mean_df = ratio_mean_df.reset_index(names=["benchmark"])
ratio_mean_df.to_csv("/home/jraskin3/vesta_csvs/NN-25-12-1-real-McCullough.csv", index=False)

**Many NN Topology Permutations**

In [ ]:
#Faithful McCullough Method
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
df = pd.read_csv(f"/home/jraskin3/vesta_csvs/32-probes-aligned-256-iters.csv").fillna(-1)
df = df[df.power <= 10**5]
df = df.drop([col for col in df.columns if '__entry' in col], axis=1)
df = df.drop([col for col in df.columns if '__return' in col], axis=1)
df = df.drop([col for col in df.columns if '__begin' in col], axis=1)
df = df.drop([col for col in df.columns if '__end' in col], axis=1)
df["cpu_power"] = (df["energy_component_2_power"] + df["energy_component_5_power"])
df["memory_power"] = (df["energy_component_0_power"] + df["energy_component_3_power"])
train_bench = "combined-benchmarks"
benchmarks = df.benchmark.unique()
ratios = {}
import tensorflow as tf


import itertools
top = [12,24,48]
out = []
for p in list(itertools.permutations(top, 2)):
    for bench in benchmarks:
        ratios[bench] = []
    for i in range(0,10):
        for component in ["power"]:
            nn_model = tf.keras.Sequential()
            nn_model.add(tf.keras.layers.Dense(24, activation="tanh"))
            nn_model.add(tf.keras.layers.Dense(p[0], activation="tanh"))
            nn_model.add(tf.keras.layers.Dense(p[1], activation="tanh"))
            nn_model.add(tf.keras.layers.Dense(1))
            nn_model.compile(optimizer="sgd", loss='mse')
            df_train, df_test = train_test_split(df, test_size=.5)
            events_df_train = df_train
            power_df_train = df_train[component]

            events_df_train = events_df_train.drop(["iteration","ts","benchmark","power","cpu_power","memory_power"], axis=1)
            events_df_train = events_df_train.drop([col for col in events_df_train.columns if 'energy_component' in col], axis=1)
            #model.fit(events_df_train, power_df_train)
            nn_model.fit(tf.convert_to_tensor(events_df_train), tf.convert_to_tensor(power_df_train), epochs=5, batch_size=256)
            for bench in benchmarks:
                events_test = df_test[df_test.benchmark == bench]
                if len(events_test) == 0:
                    continue
                events_test = events_test.drop(["iteration","ts","benchmark","power","cpu_power","memory_power"], axis=1)
                events_test = events_test.drop([col for col in events_test.columns if 'energy_component' in col], axis=1)

                #prediction_energy = model.predict(events_test)
                prediction_energy = nn_model.predict(tf.convert_to_tensor(events_test), batch_size=256)
                real_energy = df_test[df_test.benchmark == bench]["power"]
                total_prediction_energy = prediction_energy.sum()
                total_actual_energy = real_energy.sum()

                ratio =  np.abs((total_actual_energy - total_prediction_energy) / total_actual_energy)
                ratios[bench].append(ratio)
    ratio_mean = pd.DataFrame(ratios).mean()
    ratio_mean_df = pd.DataFrame(ratio_mean).rename(columns={0:"mean"})
    ratio_mean_df["std"] = pd.DataFrame(ratios).std()
    ratio_mean_df = ratio_mean_df.sort_index()
    ratio_mean_df = ratio_mean_df.reset_index(names=["benchmark"])
    out.append(f"24-{p[0]}-{p[1]}-1,{ratio_mean_df['mean'].mean()},{ratio_mean_df['mean'].std()}")

In [ ]:
for o in out:
    print(o)

**Testing Large Size Benchmarks**

In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
#dt_model = XGBRegressor(tree_method='gpu_hist', gpu_id=0)
dt_model = model
#dt_model.load_model("/home/jraskin3/vesta_csvs/xgboost_model_another_run.json")
extra_columns = pd.read_csv("/home/jraskin3/vesta_csvs/32-probes-aligned-256-iters.csv")
extra_columns = extra_columns.drop([col for col in extra_columns.columns if '__entry' in col], axis=1)
extra_columns = extra_columns.drop([col for col in extra_columns.columns if '__return' in col], axis=1)
extra_columns = extra_columns.drop([col for col in extra_columns.columns if '__begin' in col], axis=1)
extra_columns = extra_columns.drop([col for col in extra_columns.columns if '__end' in col], axis=1)
extra_columns = extra_columns.drop(["iteration","ts","benchmark","power"], axis=1)
extra_columns = extra_columns.drop([col for col in extra_columns.columns if 'energy_component' in col], axis=1)

In [ ]:
df = pd.read_csv("/home/jraskin3/jolteon_link/timur_vpc/PLDI_large_dacapo.csv")
df = df[df.power <= 10**5]
df = df.fillna(-1)

In [ ]:
for b in df.benchmark.unique():
    df_b = df[df.benchmark == b]
    ground_truth = df_b["power"].sum()
    events_df = df_b.drop(["iteration","ts","benchmark","power"], axis=1)
    events_df = events_df.drop([col for col in events_df.columns if 'energy_component' in col], axis=1)
    for e in (set(extra_columns.columns) - set(events_df.columns)):
        events_df[e] = -1
    events_df = events_df[extra_columns.columns]
    total_prediction = dt_model.predict(events_df).sum()
    error = abs((total_prediction - ground_truth)/ground_truth)
    print(b, f"{error*100:.02f}%")

In [ ]:
pmd = [1.08, .01, 2.77, 2.37, .22]
h2 = [8.16, 7.94, 6.85, 6.86, 9.20]
sunflow = [4.32, 4.16, 4.24, 4.64, 1.15]
avrora = [12.61, 13.2, 13.11, 13.16, 12.75]
batik = [2.77, 2.78, 2.57,5.63, 2.40]


In [ ]:
pmd = np.array(pmd)
h2 = np.array(h2)
sunflow = np.array(sunflow)
avrora = np.array(avrora)
batik = np.array(batik)

In [ ]:
print(f'''
    pmd = {pmd.std():.02f}, 
      h2 = {h2.std():.02f}, 
      sunflow = {sunflow.std():.02f}, 
      avrora = {avrora.std():.02f}, 
      batik = {batik.std():.02f}''')